In [ ]:
# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Qiskit Runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np
from qiskit.circuit.library import MCPhaseGate
import numpy as np
from qiskit.quantum_info import Operator

N = 3
N_tau = 31
t = 3
alpha = np.pi/10*0
jump = 1
N_reg = np.floor(np.log2(N_tau))+1
all = []
for i in range(N):
    all.append(i)

realdim = pow(2,N-2)+2
dim = pow(2,N)


j = np.arange(realdim).reshape(realdim, 1)  # Column vector of indices
k = np.arange(realdim).reshape(1, realdim)  # Row vector of indices
    
    # Compute the exponent matrix
exponent = 2 * np.pi * 1j * j * k / realdim
    
    # Compute Q matrix
DFT = np.exp(exponent) / np.sqrt(realdim)
DFTdag = np.exp(-exponent) / np.sqrt(realdim)


U = np.identity(dim, dtype=complex)
Udag = np.identity(dim, dtype=complex)
# Replace the upper-left 3x3 block with DFT_3
U[:realdim, :realdim] = DFT
Udag[:realdim, :realdim] = DFTdag


U_op = Operator(U)

Udag_op = Operator(Udag)


graph = QuantumRegister(N-1, name="graph")
meas_site = QuantumRegister(1, name="meas_site")
ancilla = QuantumRegister(1, name="ancilla")

mid_measure = ClassicalRegister(1, name="mid")
readout = ClassicalRegister(N_reg, name="final")

qc = QuantumCircuit(graph, meas_site, ancilla, mid_measure, readout)

def trial(qc,graph, meas_site, ancilla, measures, N,jump,dim, realdim,N_tau, t, N_reg, all, N_trials ):
    
    qc.unitary(U_op, all, label='DFT_3')

    eig = []
    for i in range(realdim):
        lam = 0
        for j in range(1,jump+1):
            lam += 2*np.cos(2*np.pi*i*j/realdim+alpha)
        eig.append(lam)


    binary = [0]*N #number written from sx to dx
    for i in range(realdim-1):

        num = i
        for j in reversed(range(N)):
            binary[j] = (num-num%2**j)/2**j
            num = num-binary[j]*2**j
            
        for j in range(N):
            if binary[j] ==0:
                qc.x(j)

        mc_phase_gate = MCPhaseGate(-t*eig[i], num_ctrl_qubits=N-1)
        qc.append(mc_phase_gate,all)

        for j in range(N):
            if binary[j] == 0:
                qc.x(j)

    

    for i in range(N-2):
        qc.x(i)   
    qc.x(N-1)

    mc_phase_gate = MCPhaseGate(-t*eig[realdim-2], num_ctrl_qubits=N-1)
    qc.append(mc_phase_gate,all)
    for i in range(N-2):
        qc.x(i)   
    qc.x(N-1)
    

    for i in range(N-1):
        qc.x(i)

    mc_phase_gate = MCPhaseGate(-t*eig[realdim-1], num_ctrl_qubits=N-1)
    qc.append(mc_phase_gate,all)
    for i in range(N-1):
        qc.x(i)

    
    qc.unitary(Udag_op, all, label='inv_DFT_3')
 
    qc.measure(meas_site, mid_measure)
    
    binary_array = [int(bit) for bit in bin(N_trials)[:1:-1]]
    for i in range(len(binary_array)):
        if binary_array[i] ==0:
            qc.measure(ancilla, readout[i])
        else:
            qc.x(ancilla)
            qc.measure(ancilla, readout[i])
            qc.x(ancilla)





#trial(qc,graph, meas_site, ancilla, mid_measure, N,jump,dim, realdim,N_tau, t, N_reg, all,1 )

qc.draw(output="mpl", style='iqp', cregbundle=False)